# 3.Preprocessing and Training

### Import Libaries

In [1]:
import pandas as pd
import numpy as np
import os
import pickle
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import __version__ as sklearn_version
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale
from sklearn.model_selection import train_test_split, cross_validate, GridSearchCV, learning_curve
from sklearn.preprocessing import StandardScaler, MinMaxScaler,OneHotEncoder,LabelEncoder
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import SelectKBest, f_regression


### 1.Load Data

In [2]:
datafilepath = "../data/interim/top_5_crime_data.csv"
df_top5_crime = pd.read_csv(datafilepath,index_col =0)

In [3]:
df_top5_crime.head(3)

,TIME_OCC,AREA,AREA_NAME,RPT_DIST_NO,PART_1-2,CRM_CD,CRM_CD_DESC,MOCODES,VICT_AGE,VICT_SEX,...,latitude,longitude,MTH_RPTD,DAY_RPTD,YEAR_RPTD,MTH_OCC,DAY_OCC,YEAR_OCC,ID,WEEKDAY_OCC
DATE_OCC,,,,,,,,,,,,,,,,,,,,,
2010-01-01,1200,14,Pacific,1402,1,310,BURGLARY,0344,66,F,...,34.0273,-118.4379,7,2,2010,1,1,2010,15,Friday
2010-01-01,2000,9,Van Nuys,923,1,310,BURGLARY,0344 1607,34,M,...,34.1867,-118.4640,1,1,2010,1,1,2010,19,Friday
2010-01-01,455,21,Topanga,2158,1,440,THEFT PLAIN - PETTY ($950 & UNDER),0325,0,M,...,34.1805,-118.5975,1,1,2010,1,1,2010,35,Friday


In [4]:
df_top5_crime.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1078778 entries, 2010-01-01 to 2023-08-28
Data columns (total 29 columns):
 #   Column          Non-Null Count    Dtype  
---  ------          --------------    -----  
 0   TIME_OCC        1078778 non-null  int64  
 1   AREA            1078778 non-null  int64  
 2   AREA_NAME       1078778 non-null  object 
 3   RPT_DIST_NO     1078778 non-null  int64  
 4   PART_1-2        1078778 non-null  int64  
 5   CRM_CD          1078778 non-null  int64  
 6   CRM_CD_DESC     1078778 non-null  object 
 7   MOCODES         836280 non-null   object 
 8   VICT_AGE        1078778 non-null  int64  
 9   VICT_SEX        1078778 non-null  object 
 10  VICT_DESCENT    1078778 non-null  object 
 11  PREMIS_CD       1078767 non-null  float64
 12  PREMIS_DESC     1078778 non-null  object 
 13  WEAPON_USED_CD  280775 non-null   float64
 14  WEAPON_DESC     1078778 non-null  object 
 15  STATUS          1078776 non-null  object 
 16  STATUS_DESC     1078778 non-n

In [5]:
df_top5_crime.describe()

,TIME_OCC,AREA,RPT_DIST_NO,PART_1-2,CRM_CD,VICT_AGE,PREMIS_CD,WEAPON_USED_CD,CRM_CD_1,latitude,longitude,MTH_RPTD,DAY_RPTD,YEAR_RPTD,MTH_OCC,DAY_OCC,YEAR_OCC,ID
count,1.078778e+06,1.078778e+06,1.078778e+06,1.078778e+06,1.078778e+06,1.078778e+06,1.078767e+06,280775.000000,1.078778e+06,1.078778e+06,1.078778e+06,1.078778e+06,1.078778e+06,1.078778e+06,1.078778e+06,1.078778e+06,1.078778e+06,1.078778e+06
mean,1.391752e+03,1.081540e+01,1.128016e+03,1.231463e+00,4.536655e+02,2.867120e+01,2.824912e+02,410.121353,4.536521e+02,3.404652e+01,-1.182568e+02,6.446620e+00,1.583316e+01,2.016344e+03,6.454626e+00,1.575408e+01,2.016326e+03,1.397464e+06
std,6.448947e+02,5.981725e+00,5.982331e+02,4.217676e-01,1.187441e+02,2.244701e+01,2.171584e+02,42.936659,1.187468e+02,9.946287e-01,3.433892e+00,3.424897e+00,8.790109e+00,3.957563e+00,3.427069e+00,8.808094e+00,3.955827e+00,8.280711e+05
min,1.000000e+00,1.000000e+00,1.000000e+02,1.000000e+00,3.100000e+02,-1.000000e+01,1.010000e+02,101.000000,-1.000000e+00,0.000000e+00,-1.188279e+02,1.000000e+00,1.000000e+00,2.010000e+03,1.000000e+00,1.000000e+00,2.010000e+03,1.500000e+01
25%,9.450000e+02,6.000000e+00,6.360000e+02,1.000000e+00,3.300000e+02,0.000000e+00,1.010000e+02,400.000000,3.300000e+02,3.401240e+01,-1.184322e+02,4.000000e+00,8.000000e+00,2.013000e+03,4.000000e+00,8.000000e+00,2.013000e+03,6.715352e+05
50%,1.500000e+03,1.100000e+01,1.148000e+03,1.000000e+00,4.400000e+02,2.900000e+01,1.220000e+02,400.000000,4.400000e+02,3.406070e+01,-1.183279e+02,6.000000e+00,1.600000e+01,2.016000e+03,6.000000e+00,1.600000e+01,2.016000e+03,1.381670e+06
75%,1.915000e+03,1.600000e+01,1.612000e+03,1.000000e+00,5.100000e+02,4.500000e+01,5.010000e+02,400.000000,5.100000e+02,3.416670e+01,-1.182760e+02,9.000000e+00,2.300000e+01,2.020000e+03,9.000000e+00,2.300000e+01,2.020000e+03,2.116268e+06
max,2.359000e+03,2.100000e+01,2.199000e+03,2.000000e+00,6.240000e+02,1.180000e+02,9.710000e+02,516.000000,6.490000e+02,3.479070e+01,0.000000e+00,1.200000e+01,3.100000e+01,2.023000e+03,1.200000e+01,3.100000e+01,2.023000e+03,2.865898e+06


In [6]:
df_top5_crime.shape

(1078778, 29)

### 2.Identify and convert categorical variables into dummy/indicator features.

In [7]:
# Identify categorical columns to convert to dummy variables
categorical_cols = df_top5_crime.select_dtypes(include=['object', 'category', 'bool']).columns.tolist()
print(categorical_cols)

['AREA_NAME', 'CRM_CD_DESC', 'MOCODES', 'VICT_SEX', 'VICT_DESCENT', 'PREMIS_DESC', 'WEAPON_DESC', 'STATUS', 'STATUS_DESC', 'LOCATION', 'WEEKDAY_OCC']


<p>After re-looking the columns list.I decided to remove the unnessary columns before encoding ('MOCODES','VICT_DESCENT', 'PREMIS_DESC', 'WEAPON_DESC', 'STATUS', 'STATUS_DESC')  Since modeling will be focus on  'AREA_NAME', 'CRM_CD_DESC','WEEKDAY_OCC'.</p> </br>
<p>ID column will be also removed and reinstate after cleaning duplicate.</p>

In [8]:
columns_to_drop = ['ID','MOCODES','VICT_DESCENT', 'PREMIS_DESC', 'WEAPON_DESC', 'STATUS', 'STATUS_DESC','LOCATION']
df_top5_crime.drop(columns=columns_to_drop, axis=1, inplace=True)

In [9]:
categorical_cols = df_top5_crime.select_dtypes(include=['object', 'category', 'bool']).columns.tolist()
print(categorical_cols)

['AREA_NAME', 'CRM_CD_DESC', 'VICT_SEX', 'WEEKDAY_OCC']


<b>Encoding categorical variables like AREA_NAME and WEEKDAY_OCCRANCE since machine learning models require numerical input.Encode main categorical variables to be used in modeling later.</b>

In [10]:
#Encode categorical variables
encoder = LabelEncoder()
df_top5_crime['AREA_NAME_Encoded'] = encoder.fit_transform(df_top5_crime['AREA_NAME'])
df_top5_crime['WEEKDAY_OCC_Encoded'] = encoder.fit_transform(df_top5_crime['WEEKDAY_OCC'])
df_top5_crime['CRM_CD_DESC_Encoded'] = encoder.fit_transform(df_top5_crime['CRM_CD_DESC'])
df_top5_crime['VICT_SEX_Encoded'] = encoder.fit_transform(df_top5_crime['VICT_SEX'])

Categorical columns are converted to string type for consistency before applying one-hot encoding.

In [11]:
# Convert all categorical columns to strings to ensure consistency
df_top5_crime[categorical_cols] = df_top5_crime[categorical_cols].astype(str)

In [12]:
df_top5_crime.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1078778 entries, 2010-01-01 to 2023-08-28
Data columns (total 25 columns):
 #   Column               Non-Null Count    Dtype  
---  ------               --------------    -----  
 0   TIME_OCC             1078778 non-null  int64  
 1   AREA                 1078778 non-null  int64  
 2   AREA_NAME            1078778 non-null  object 
 3   RPT_DIST_NO          1078778 non-null  int64  
 4   PART_1-2             1078778 non-null  int64  
 5   CRM_CD               1078778 non-null  int64  
 6   CRM_CD_DESC          1078778 non-null  object 
 7   VICT_AGE             1078778 non-null  int64  
 8   VICT_SEX             1078778 non-null  object 
 9   PREMIS_CD            1078767 non-null  float64
 10  WEAPON_USED_CD       280775 non-null   float64
 11  CRM_CD_1             1078778 non-null  float64
 12  latitude             1078778 non-null  float64
 13  longitude            1078778 non-null  float64
 14  MTH_RPTD             1078778 non-null  int6

<p>Check if there is any duplicate records after removing raw data unique IDs .</p>

In [13]:
#Remove dupliate and keep as new cleaned dataframes
df_top5_crime= df_top5_crime.drop_duplicates(keep='last')

In [14]:
df_top5_crime.shape

(1077551, 25)

<p>Add ID sort by index</p>

In [15]:
# Sort DataFrame by date column
df_top5_crime = df_top5_crime.sort_values(by='DATE_OCC')

# Reset index if needed
df_top5_crime = df_top5_crime.reset_index(drop=True)

# Add index+1 as ID column
# df_top5_crime['ID'] = df_top5_crime.index + 1

One-hot encoding is applied, transforming categorical variables into a set of binary columns, each representing a category's presence or absence.

In [16]:
# Apply one-hot encoding to categorical variables
ohe = OneHotEncoder(sparse=False, handle_unknown='ignore')
df_dummies = pd.DataFrame(ohe.fit_transform(df_top5_crime[categorical_cols]), columns=ohe.get_feature_names(categorical_cols))

In [17]:
df_dummies.head(5)

,AREA_NAME_77th Street,AREA_NAME_Central,AREA_NAME_Devonshire,AREA_NAME_Foothill,AREA_NAME_Harbor,AREA_NAME_Hollenbeck,AREA_NAME_Hollywood,AREA_NAME_Mission,AREA_NAME_N Hollywood,AREA_NAME_Newton,...,VICT_SEX_N,VICT_SEX_Unknown,VICT_SEX_X,WEEKDAY_OCC_Friday,WEEKDAY_OCC_Monday,WEEKDAY_OCC_Saturday,WEEKDAY_OCC_Sunday,WEEKDAY_OCC_Thursday,WEEKDAY_OCC_Tuesday,WEEKDAY_OCC_Wednesday
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [18]:
# Drop the original categorical columns 
#! df_top5_crime = df_top5_crime.drop(categorical_cols, axis=1)

# concatenate the new dummy variables
df_top5_crime = pd.concat([df_top5_crime, df_dummies], axis=1)

In [19]:
df_top5_crime.shape

(1077551, 64)

In [20]:
df_top5_crime.head(3)

,TIME_OCC,AREA,AREA_NAME,RPT_DIST_NO,PART_1-2,CRM_CD,CRM_CD_DESC,VICT_AGE,VICT_SEX,PREMIS_CD,...,VICT_SEX_N,VICT_SEX_Unknown,VICT_SEX_X,WEEKDAY_OCC_Friday,WEEKDAY_OCC_Monday,WEEKDAY_OCC_Saturday,WEEKDAY_OCC_Sunday,WEEKDAY_OCC_Thursday,WEEKDAY_OCC_Tuesday,WEEKDAY_OCC_Wednesday
0,1200,14,Pacific,1402,1,310,BURGLARY,66,F,501.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1900,3,Southwest,321,1,440,THEFT PLAIN - PETTY ($950 & UNDER),26,M,203.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1130,3,Southwest,321,1,440,THEFT PLAIN - PETTY ($950 & UNDER),26,M,203.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


### 3.Standardize the magnitude of numeric features using a scaler.

In [21]:
# Identify numerical columns for scaling
numerical_cols = df_top5_crime.select_dtypes(include=['int64', 'float64']).columns.tolist()
print (numerical_cols)

['TIME_OCC', 'AREA', 'RPT_DIST_NO', 'PART_1-2', 'CRM_CD', 'VICT_AGE', 'PREMIS_CD', 'WEAPON_USED_CD', 'CRM_CD_1', 'latitude', 'longitude', 'MTH_RPTD', 'DAY_RPTD', 'YEAR_RPTD', 'MTH_OCC', 'DAY_OCC', 'YEAR_OCC', 'AREA_NAME_Encoded', 'WEEKDAY_OCC_Encoded', 'CRM_CD_DESC_Encoded', 'VICT_SEX_Encoded', 'AREA_NAME_77th Street', 'AREA_NAME_Central', 'AREA_NAME_Devonshire', 'AREA_NAME_Foothill', 'AREA_NAME_Harbor', 'AREA_NAME_Hollenbeck', 'AREA_NAME_Hollywood', 'AREA_NAME_Mission', 'AREA_NAME_N Hollywood', 'AREA_NAME_Newton', 'AREA_NAME_Northeast', 'AREA_NAME_Olympic', 'AREA_NAME_Pacific', 'AREA_NAME_Rampart', 'AREA_NAME_Southeast', 'AREA_NAME_Southwest', 'AREA_NAME_Topanga', 'AREA_NAME_Van Nuys', 'AREA_NAME_West LA', 'AREA_NAME_West Valley', 'AREA_NAME_Wilshire', 'CRM_CD_DESC_BATTERY - SIMPLE ASSAULT', 'CRM_CD_DESC_BURGLARY', 'CRM_CD_DESC_BURGLARY FROM VEHICLE', 'CRM_CD_DESC_THEFT PLAIN - PETTY ($950 & UNDER)', 'CRM_CD_DESC_VEHICLE - STOLEN', 'VICT_SEX_F', 'VICT_SEX_H', 'VICT_SEX_M', 'VICT_SEX_N

Numerical columns are standardized using StandardScaler to ensure uniformity in their magnitude, which is crucial for many machine learning algorithms.

In [22]:
# Apply standard scaling to numerical features
scaler = StandardScaler()
df_top5_crime[numerical_cols] = scaler.fit_transform(df_top5_crime[numerical_cols])

In [23]:
# Combine with numeric data
df_numeric = df_top5_crime[numerical_cols]
df_processed = pd.concat([df_numeric, df_dummies], axis=1)

In [24]:
df_numeric.head(3)

,TIME_OCC,AREA,RPT_DIST_NO,PART_1-2,CRM_CD,VICT_AGE,PREMIS_CD,WEAPON_USED_CD,CRM_CD_1,latitude,...,VICT_SEX_N,VICT_SEX_Unknown,VICT_SEX_X,WEEKDAY_OCC_Friday,WEEKDAY_OCC_Monday,WEEKDAY_OCC_Saturday,WEEKDAY_OCC_Sunday,WEEKDAY_OCC_Thursday,WEEKDAY_OCC_Tuesday,WEEKDAY_OCC_Wednesday
0,-0.297526,0.532446,0.458049,-0.548261,-1.209521,1.662360,1.006507,NaN,-1.209380,-0.019322,...,-0.002725,-0.533766,-0.167708,2.326996,-0.401727,-0.413483,-0.396332,-0.407086,-0.402139,-0.406866
1,0.788060,-1.306509,-1.348962,-0.548261,-0.114449,-0.119481,-0.365924,NaN,-0.114334,-0.022439,...,-0.002725,-0.533766,-0.167708,2.326996,-0.401727,-0.413483,-0.396332,-0.407086,-0.402139,-0.406866
2,-0.406085,-1.306509,-1.348962,-0.548261,-0.114449,-0.119481,-0.365924,NaN,-0.114334,-0.022439,...,-0.002725,-0.533766,-0.167708,2.326996,-0.401727,-0.413483,-0.396332,-0.407086,-0.402139,-0.406866


In [25]:
df_processed.head(3)

,TIME_OCC,AREA,RPT_DIST_NO,PART_1-2,CRM_CD,VICT_AGE,PREMIS_CD,WEAPON_USED_CD,CRM_CD_1,latitude,...,VICT_SEX_N,VICT_SEX_Unknown,VICT_SEX_X,WEEKDAY_OCC_Friday,WEEKDAY_OCC_Monday,WEEKDAY_OCC_Saturday,WEEKDAY_OCC_Sunday,WEEKDAY_OCC_Thursday,WEEKDAY_OCC_Tuesday,WEEKDAY_OCC_Wednesday
0,-0.297526,0.532446,0.458049,-0.548261,-1.209521,1.662360,1.006507,NaN,-1.209380,-0.019322,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.788060,-1.306509,-1.348962,-0.548261,-0.114449,-0.119481,-0.365924,NaN,-0.114334,-0.022439,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,-0.406085,-1.306509,-1.348962,-0.548261,-0.114449,-0.119481,-0.365924,NaN,-0.114334,-0.022439,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [26]:
# Apply standard scaling to the numerical features
scaler = StandardScaler()
df_processed[numerical_cols] = scaler.fit_transform(df_processed[numerical_cols])

In [27]:
df_processed.head(3)

,TIME_OCC,AREA,RPT_DIST_NO,PART_1-2,CRM_CD,VICT_AGE,PREMIS_CD,WEAPON_USED_CD,CRM_CD_1,latitude,...,VICT_SEX_N,VICT_SEX_Unknown,VICT_SEX_X,WEEKDAY_OCC_Friday,WEEKDAY_OCC_Monday,WEEKDAY_OCC_Saturday,WEEKDAY_OCC_Sunday,WEEKDAY_OCC_Thursday,WEEKDAY_OCC_Tuesday,WEEKDAY_OCC_Wednesday
0,-0.297526,0.532446,0.458049,-0.548261,-1.209521,1.662360,1.006507,NaN,-1.209380,-0.019322,...,-0.002725,-0.533766,-0.167708,2.326996,-0.401727,-0.413483,-0.396332,-0.407086,-0.402139,-0.406866
1,0.788060,-1.306509,-1.348962,-0.548261,-0.114449,-0.119481,-0.365924,NaN,-0.114334,-0.022439,...,-0.002725,-0.533766,-0.167708,2.326996,-0.401727,-0.413483,-0.396332,-0.407086,-0.402139,-0.406866
2,-0.406085,-1.306509,-1.348962,-0.548261,-0.114449,-0.119481,-0.365924,NaN,-0.114334,-0.022439,...,-0.002725,-0.533766,-0.167708,2.326996,-0.401727,-0.413483,-0.396332,-0.407086,-0.402139,-0.406866


In [28]:
df_processed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1077551 entries, 0 to 1077550
Data columns (total 99 columns):
 #   Column                                          Non-Null Count    Dtype  
---  ------                                          --------------    -----  
 0   TIME_OCC                                        1077551 non-null  float64
 1   AREA                                            1077551 non-null  float64
 2   RPT_DIST_NO                                     1077551 non-null  float64
 3   PART_1-2                                        1077551 non-null  float64
 4   CRM_CD                                          1077551 non-null  float64
 5   VICT_AGE                                        1077551 non-null  float64
 6   PREMIS_CD                                       1077540 non-null  float64
 7   WEAPON_USED_CD                                  280096 non-null   float64
 8   CRM_CD_1                                        1077551 non-null  float64
 9   latitude     

Next Step is splitting the dataset into features (X) and the target variable (y), which in this case is CRM_CD_DESC. The dataset is split into feature (X) and target (y) sets.

In [29]:
df_top5_crime.head()

,TIME_OCC,AREA,AREA_NAME,RPT_DIST_NO,PART_1-2,CRM_CD,CRM_CD_DESC,VICT_AGE,VICT_SEX,PREMIS_CD,...,VICT_SEX_N,VICT_SEX_Unknown,VICT_SEX_X,WEEKDAY_OCC_Friday,WEEKDAY_OCC_Monday,WEEKDAY_OCC_Saturday,WEEKDAY_OCC_Sunday,WEEKDAY_OCC_Thursday,WEEKDAY_OCC_Tuesday,WEEKDAY_OCC_Wednesday
0,-0.297526,0.532446,Pacific,0.458049,-0.548261,-1.209521,BURGLARY,1.662360,F,1.006507,...,-0.002725,-0.533766,-0.167708,2.326996,-0.401727,-0.413483,-0.396332,-0.407086,-0.402139,-0.406866
1,0.788060,-1.306509,Southwest,-1.348962,-0.548261,-0.114449,THEFT PLAIN - PETTY ($950 & UNDER),-0.119481,M,-0.365924,...,-0.002725,-0.533766,-0.167708,2.326996,-0.401727,-0.413483,-0.396332,-0.407086,-0.402139,-0.406866
2,-0.406085,-1.306509,Southwest,-1.348962,-0.548261,-0.114449,THEFT PLAIN - PETTY ($950 & UNDER),-0.119481,M,-0.365924,...,-0.002725,-0.533766,-0.167708,2.326996,-0.401727,-0.413483,-0.396332,-0.407086,-0.402139,-0.406866
3,0.555435,-1.306509,Southwest,-1.372364,1.823947,1.435499,BATTERY - SIMPLE ASSAULT,-0.030389,F,1.006507,...,-0.002725,-0.533766,-0.167708,2.326996,-0.401727,-0.413483,-0.396332,-0.407086,-0.402139,-0.406866
4,0.788060,0.198091,77th Street,0.235725,-0.548261,-1.041048,BURGLARY FROM VEHICLE,-0.030389,F,-0.835683,...,-0.002725,-0.533766,-0.167708,2.326996,-0.401727,-0.413483,-0.396332,-0.407086,-0.402139,-0.406866


In [30]:
target_var_cols = ['VICT_AGE', 'latitude', 'longitude', 'MTH_RPTD', 'DAY_RPTD', 'YEAR_RPTD', 'MTH_OCC', 'DAY_OCC', 'YEAR_OCC','WEEKDAY_OCC_Encoded','AREA_NAME_Encoded']

#!X = df_top5_crime.drop(columns=target_var_cols, axis=1)
#!y = df_top5_crime[target_var_cols]

# Splitting the dataset into features and target variable
X = df_top5_crime[target_var_cols]
y = df_top5_crime['CRM_CD_DESC']

In [31]:
print (X)

         VICT_AGE  latitude  longitude  MTH_RPTD  DAY_RPTD  YEAR_RPTD  \
0        1.662360 -0.019322  -0.052739  0.161585 -1.573709  -1.602962   
1       -0.119481 -0.022439  -0.033607 -1.590315 -1.687476  -1.602962   
2       -0.119481 -0.022439  -0.033607 -1.590315 -1.687476  -1.602962   
3       -0.030389 -0.012888  -0.022599 -1.590315 -1.687476  -1.602962   
4       -0.030389 -0.084069  -0.001166 -1.590315 -1.346177  -1.602962   
...           ...       ...        ...       ...       ...        ...   
1077546 -1.277677  0.157223  -0.098954  0.453568  1.384212   1.681918   
1077547 -1.277677 -0.049785  -0.015901  0.453568  1.384212   1.681918   
1077548 -1.277677 -0.092413  -0.004981  0.453568  1.384212   1.681918   
1077549  0.459617  0.071565  -0.010485  0.453568  1.384212   1.681918   
1077550 -1.277677 -0.062755  -0.012669  0.453568  1.384212   1.681918   

          MTH_OCC   DAY_OCC  YEAR_OCC  WEEKDAY_OCC_Encoded  AREA_NAME_Encoded  
0       -1.591655 -1.675067 -1.599046      

In [32]:
print (y)

0                                    BURGLARY
1          THEFT PLAIN - PETTY ($950 & UNDER)
2          THEFT PLAIN - PETTY ($950 & UNDER)
3                    BATTERY - SIMPLE ASSAULT
4                       BURGLARY FROM VEHICLE
                          ...                
1077546              BATTERY - SIMPLE ASSAULT
1077547                              BURGLARY
1077548              BATTERY - SIMPLE ASSAULT
1077549                 BURGLARY FROM VEHICLE
1077550                      VEHICLE - STOLEN
Name: CRM_CD_DESC, Length: 1077551, dtype: object


### 4.Split the dataset into training and testing datasets.

The dataset is further split into training and testing sets, following the common practice of holding out a portion of the data for model evaluation.

In [33]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [34]:
# Output the shapes of the resulting dataframes
(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

((862040, 11), (215511, 11), (862040,), (215511,))

Save the data for next step , Modeling.

In [35]:
##### saving as Pickle
import pickle

save_datafilepath = "../data/interim/train_test_split.pkl"
with open(save_datafilepath, 'wb') as file:
    pickle.dump([X,y,X_train, X_test, y_train, y_test], file)